# TRAIN DB-StyleGAN

In [4]:
# !bash ./docker_run.sh python train_DB_StyleGAN2.py --cfg="auto" --snap=20  --data="DATASETS/VIS-TH_Eurocom_256x256_only_normal/images" --resume="ffhq256" --gpus=1 --mirror=1 --outdir="IJCB_EXPERIMENTS/DB_SG2/EXPERIMENT_VIS-TH_Eurocom_256x256_only_normal_NO_VALID"

# !bash ./docker_run.sh python train_DB_StyleGAN2.py --cfg="auto" --snap=20  --data="DATASETS/Oulu_CASIA_VIS_NIR_256x256/images" --resume="ffhq256" --gpus=1 --mirror=1 --outdir="IJCB_EXPERIMENTS/DB_SG2/EXPERIMENT_Oulu_CASIA_VIS_NIR_256x256_only_strong"

#!bash ./docker_run.sh python train_DB_StyleGAN2.py --cfg="auto" --snap=20  --data="DATASETS/Tufts_256_center/images" --resume="ffhq256" --gpus=1 --mirror=1 --outdir="EXPERIMENTS/DB_SG2/EXPERIMENT_Tufts_256_center"

# Batch 8 
# !bash ./docker_run.sh python train_DB_StyleGAN2.py --cfg="auto" --snap=20 --gpus=1 --mirror=1 --GPU_DEVICE_NUMBER=0 --batch=12  --data="DATASETS/Tufts_256_center/train/images" --resume="ffhq256"  --outdir="EXPERIMENTS/DB_SG2/EXPERIMENT_tufts_256_poses_3-5"

!bash ./docker_run.sh python train_DB_StyleGAN2.py --cfg="auto" --snap=20 --gpus=1 --mirror=1 --GPU_DEVICE_NUMBER=0 --batch=12  --data="DATASETS/tufts_256_poses_1-7_aligned/train/images" --resume="ffhq256"  --outdir="EXPERIMENTS/DB_SG2/EXPERIMENT_tufts_256_poses_1-7_aligned_loss_one_tenth"

# resume from tenth
#!bash ./docker_run.sh python train_DB_StyleGAN2.py --cfg="auto" --snap=20 --gpus=1 --mirror=1 --GPU_DEVICE_NUMBER=0 --batch=12  --data="DATASETS/tufts_256_poses_1-7_aligned/train/images" --resume="EXPERIMENTS/checkpoints/FRAIS_2022_checkpoints/Tufts_1-7_aligned_one_tenth-001683.pkl"  --outdir="EXPERIMENTS/DB_SG2/EXPERIMENT_tufts_256_poses_1-7_aligned_loss_one_tenth_continue_equal"




Training options:
{
  "num_gpus": 1,
  "GPU_DEVICE_NUMBER": 0,
  "image_snapshot_ticks": 20,
  "network_snapshot_ticks": 20,
  "metrics": [
    "fid50k_full"
  ],
  "random_seed": 0,
  "training_set_kwargs": {
    "class_name": "training_scripts_DB_SG2.dataset.ImageFolderDataset",
    "path": "DATASETS/tufts_256_poses_1-7_aligned/train/images",
    "use_labels": false,
    "max_size": 1965,
    "xflip": true,
    "resolution": 256
  },
  "data_loader_kwargs": {
    "pin_memory": true,
    "num_workers": 3,
    "prefetch_factor": 2
  },
  "G_kwargs": {
    "class_name": "training_scripts_DB_SG2.networks.Generator",
    "z_dim": 512,
    "w_dim": 512,
    "mapping_kwargs": {
      "num_layers": 8
    },
    "synthesis_kwargs": {
      "channel_base": 16384,
      "channel_max": 512,
      "num_fp16_res": 4,
      "conv_clamp": 256
    }
  },
  "D_kwargs": {
    "class_name": "training_scripts_DB_SG2.networks.Discriminator",
    "block_kwargs": {},
    "mapping_kwargs": {},
    "epilogue

# Train and Use Style Interpreter

## 1. Generate Training Images with DB-SG2 

In [1]:
# path = "CrossEyed_NIR_RGB_new_1120"
path = "TUFTS_1-7_nonaligned_cfg_256"

exp = "EXPERIMENTS/interpreter/FRAIS_2022/" + path
sv_path = "EXPERIMENTS/interpreter/FRAIS_2022/" + path



In [2]:
exp_generate = exp + "/generate.json" 

samples = 100 # 100
# GENERATE training images
!bash docker_run.sh python make_training_data_DB_SG2.py --exp=$exp_generate --sv_path=$sv_path --num_sample=$samples



Traceback (most recent call last):
  File "make_training_data_DB_SG2.py", line 285, in <module>
    opts = json.load(open(args.exp, 'r'))
FileNotFoundError: [Errno 2] No such file or directory: 'EXPERIMENTS/interpreter/FRAIS_2022/TUFTS_1-7_nonaligned_cfg_256/generate.json'


## Intermediate Steps

### 1.1 Annotate the Generated Images as written in the instructions
### 1.2 Rename latent_stylegan to back_latent_stylegan 
### 1.3 Run preprocessing script (or use your own preprocessing to obtain the required files)

## 2. Train Style Interpreter on Annotated Data 

First create generate and train json files based on the templates

In [2]:
exp_interpreter = exp + "/train_datagan.json" 
!bash docker_run.sh python train_interpreter_DB_SG2.py --exp $exp_interpreter

Device: cuda:0
Opt {'exp_dir': 'IJCB_EXPERIMENTS/interpreter/CrossEyed_NIR_RGB_new_1120_2', 'batch_size': 64, 'category': 'eyes_256', 'debug': False, 'dim': [256, 256, 4992], 'deeplab_res': 256, 'number_class': 4, 'testing_data_number_class': 4, 'max_training': 2, 'stylegan_ver': '2', 'annotation_data_from_w': False, 'annotation_mask_path': 'eyes_processed', 'testing_path': '', 'average_latent': 'avg_latent_stylegan.npy', 'annotation_image_latent_path': 'latent_stylegan.npy', 'stylegan_checkpoint': 'IJCB_EXPERIMENTS/checkpoints/IJCB_2022_checkpoints/CrossEyed_NIR_RGB_NEW_001120.pkl', 'model_num': 10, 'upsample_mode': 'bilinear', 'upsample_threshold': '8'}
cp: 'IJCB_EXPERIMENTS/interpreter/CrossEyed_NIR_RGB_new_1120_2/train_datagan.json' and 'IJCB_EXPERIMENTS/interpreter/CrossEyed_NIR_RGB_new_1120_2/train_datagan.json' are the same file
Prepare data
-- Prepare stylegan
Eyes category
---- Resolution: 256  Layers: 7
---- Get avg latent
---- Latent to torch
AVG latent torch.Size([14, 512])

## 3. Generate Data (RGB, NIR, MASK)

In [3]:
resume_model = "IJCB_EXPERIMENTS/interpreter/CrossEyed_NIR_RGB_new_1120_2"
num_samples = 5000

exp_interpreter = exp + "/train_datagan.json" 
!bash docker_run.sh python train_interpreter_DB_SG2.py --generate_data True --exp=$exp_interpreter  --resume $resume_model  --num_sample=$num_samples


Device: cuda:0
Opt {'exp_dir': 'IJCB_EXPERIMENTS/interpreter/CrossEyed_NIR_RGB_new_1120_2', 'batch_size': 64, 'category': 'eyes_256', 'debug': False, 'dim': [256, 256, 4992], 'deeplab_res': 256, 'number_class': 4, 'testing_data_number_class': 4, 'max_training': 2, 'stylegan_ver': '2', 'annotation_data_from_w': False, 'annotation_mask_path': 'eyes_processed', 'testing_path': '', 'average_latent': 'avg_latent_stylegan.npy', 'annotation_image_latent_path': 'latent_stylegan.npy', 'stylegan_checkpoint': 'IJCB_EXPERIMENTS/checkpoints/IJCB_2022_checkpoints/CrossEyed_NIR_RGB_NEW_001120.pkl', 'model_num': 10, 'upsample_mode': 'bilinear', 'upsample_threshold': '8'}
cp: 'IJCB_EXPERIMENTS/interpreter/CrossEyed_NIR_RGB_new_1120_2/train_datagan.json' and 'IJCB_EXPERIMENTS/interpreter/CrossEyed_NIR_RGB_new_1120_2/train_datagan.json' are the same file
Generate data
Experiment folder created at: IJCB_EXPERIMENTS/interpreter/CrossEyed_NIR_RGB_new_1120_2/samples_5000
Eyes category
---- Resolution: 256  L

In [5]:
#resume_model = "IJCB_EXPERIMENTS/interpreter/CrossEyed_NIR_RGB_new_1120"
num_samples = 12

exp_interpreter = exp + "/train_datagan.json" 
!bash docker_run.sh python train_interpreter_DB_SG2.py --generate_data True --exp=$exp_interpreter  --resume $resume_model  --num_sample=$num_samples

Device: cuda:0
Opt {'exp_dir': 'IJCB_EXPERIMENTS/interpreter/CrossEyed_NIR_RGB_new_1120', 'batch_size': 64, 'category': 'eyes_256', 'debug': False, 'dim': [256, 256, 4992], 'deeplab_res': 256, 'number_class': 4, 'testing_data_number_class': 4, 'max_training': 8, 'stylegan_ver': '2', 'annotation_data_from_w': False, 'annotation_mask_path': 'eyes_processed', 'testing_path': '', 'average_latent': 'avg_latent_stylegan.npy', 'annotation_image_latent_path': 'latent_stylegan.npy', 'stylegan_checkpoint': 'IJCB_EXPERIMENTS/checkpoints/IJCB_2022_checkpoints/CrossEyed_NIR_RGB_NEW_001120.pkl', 'model_num': 10, 'upsample_mode': 'bilinear', 'upsample_threshold': '8'}
cp: 'IJCB_EXPERIMENTS/interpreter/CrossEyed_NIR_RGB_new_1120/train_datagan.json' and 'IJCB_EXPERIMENTS/interpreter/CrossEyed_NIR_RGB_new_1120/train_datagan.json' are the same file
Generate data
Eyes category
---- Resolution: 256  Layers: 7
---- Get avg latent
---- Latent to torch
AVG latent torch.Size([14, 512])
----  Build Generator
Re